<a href="https://colab.research.google.com/github/strg-frontier/pytorch_wine_quality/blob/master/resnet_%E8%BB%A2%E7%A7%BB%E5%AD%A6%E7%BF%92%E6%AF%94%E8%BC%83.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import torchvision.models as models
from torchvision.datasets import ImageFolder
from torchvision import transforms

from tqdm import tqdm

In [8]:
from google.colab import files
uploaded = files.upload()

Saving taco_and_burrito.tar.gz to taco_and_burrito.tar.gz


In [0]:
!tar -xzf taco_and_burrito.tar.gz

In [10]:
!ls taco_and_burrito

test  train


000.jpg  040.jpg  080.jpg  120.jpg  160.jpg  200.jpg  240.jpg  281.jpg	321.jpg
001.jpg  041.jpg  081.jpg  121.jpg  161.jpg  201.jpg  241.jpg  282.jpg	322.jpg
002.jpg  042.jpg  082.jpg  122.jpg  162.jpg  202.jpg  242.jpg  283.jpg	323.jpg
003.jpg  043.jpg  083.jpg  123.jpg  163.jpg  203.jpg  243.jpg  284.jpg	324.jpg
004.jpg  044.jpg  084.jpg  124.jpg  164.jpg  204.jpg  244.jpg  285.jpg	325.jpg
005.jpg  045.jpg  085.jpg  125.jpg  165.jpg  205.jpg  245.jpg  286.jpg	326.jpg
006.jpg  046.jpg  086.jpg  126.jpg  166.jpg  206.jpg  246.jpg  287.jpg	327.jpg
007.jpg  047.jpg  087.jpg  127.jpg  167.jpg  207.jpg  247.jpg  288.jpg	328.jpg
008.jpg  048.jpg  088.jpg  128.jpg  168.jpg  208.jpg  248.jpg  289.jpg	329.jpg
009.jpg  049.jpg  089.jpg  129.jpg  169.jpg  209.jpg  249.jpg  290.jpg	330.jpg
010.jpg  050.jpg  090.jpg  130.jpg  170.jpg  210.jpg  250.jpg  291.jpg	331.jpg
011.jpg  051.jpg  091.jpg  131.jpg  171.jpg  211.jpg  251.jpg  292.jpg	332.jpg
012.jpg  052.jpg  092.jpg  132.jpg  172.jpg  212.jpg

In [0]:
from IPython.display import Image,display_jpeg
display_jpeg(Image('./taco_and_burrito/train/burrito/007.jpg'))
display_jpeg(Image('./taco_and_burrito/train/taco/027.jpg'))

In [0]:
base_dir = './taco_and_burrito'
train_imgs = ImageFolder(base_dir + '/train/',
                         transform=transforms.Compose([
                             transforms.RandomCrop(224),
                             transforms.ToTensor()]
                         ))
test_imgs = ImageFolder(base_dir + '/test/',
                         transform=transforms.Compose([
                             transforms.RandomCrop(224),
                             transforms.ToTensor()]
                         ))

train_loader = DataLoader(
    train_imgs, batch_size=32, shuffle=True)
test_loader = DataLoader(
    test_imgs, batch_size=32, shuffle=False)

In [12]:
train_imgs.classes

['burrito', 'taco']

In [13]:
# 転移学習。最後の全結合のみ
resnet = models.resnet18(pretrained=True)

for p in resnet.parameters():
    p.requires_grad=False

fc_input_dim = resnet.fc.in_features
resnet.fc = nn.Linear(fc_input_dim, 2)
resnet

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [19]:
import datetime

def train(net, epoch):
    # 最適化手法のパラメータ設定
    optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
    # loss関数の定義
    criterion = nn.CrossEntropyLoss()

    total_loss = 0
    total_size = 0
    net.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        #data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = net(data)
        loss = criterion(output, target)
        total_loss += loss.item()
        total_size += data.size(0)
        loss.backward()
        optimizer.step()
        if batch_idx % 1000 == 0:
            now = datetime.datetime.now()
            print('[{}] Train Epoch: {} [{}/{} ({:.0f}%)]\tAverage loss: {:.6f}'.format(
                now,
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), total_loss / total_size))
    return net

for epoch in range(1, 10 + 1):
    resnet = train(resnet, epoch)

[2020-02-16 13:00:57.174042] Train Epoch: 1 [0/712 (0%)]	Average loss: 0.458143
[2020-02-16 13:01:52.341199] Train Epoch: 2 [0/712 (0%)]	Average loss: 0.056315
[2020-02-16 13:02:46.915794] Train Epoch: 3 [0/712 (0%)]	Average loss: 0.151480
[2020-02-16 13:03:41.484012] Train Epoch: 4 [0/712 (0%)]	Average loss: 0.156596
[2020-02-16 13:04:35.798740] Train Epoch: 5 [0/712 (0%)]	Average loss: 0.012054
[2020-02-16 13:05:30.307138] Train Epoch: 6 [0/712 (0%)]	Average loss: 0.013623
[2020-02-16 13:06:24.828741] Train Epoch: 7 [0/712 (0%)]	Average loss: 0.052272
[2020-02-16 13:07:19.466125] Train Epoch: 8 [0/712 (0%)]	Average loss: 0.104411
[2020-02-16 13:08:13.812162] Train Epoch: 9 [0/712 (0%)]	Average loss: 0.479038
[2020-02-16 13:09:08.074847] Train Epoch: 10 [0/712 (0%)]	Average loss: 0.104775


In [20]:
resnet = resnet.eval()
pred = []
Y = []
for i, (x,y) in enumerate(test_loader):
    with torch.no_grad():
        output = resnet(x)
    pred += [int(l.argmax()) for l in output]
    Y += [int(l) for l in y]
correct=0
for i in range(len(Y)):
    correct += Y[i]==pred[i]
print(correct/len(Y))

0.7833333333333333


In [0]:
# 最後の全結合層に加え、その前の畳み込み層２つも学習
resnet2 = models.resnet18(pretrained=True)
for p in resnet2.parameters():
    p.requires_grad=False

fc_input_dim = resnet2.fc.in_features
resnet2.fc = nn.Linear(fc_input_dim, 2)

resnet2.layer4[1].conv1 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
resnet2.layer4[1].conv2 = nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)

In [22]:
for epoch in range(1, 10 + 1):
    resnet2 = train(resnet2, epoch)

[2020-02-16 13:10:08.566019] Train Epoch: 1 [0/712 (0%)]	Average loss: 0.026837
[2020-02-16 13:11:11.957103] Train Epoch: 2 [0/712 (0%)]	Average loss: 0.044932
[2020-02-16 13:12:15.149604] Train Epoch: 3 [0/712 (0%)]	Average loss: 0.060041
[2020-02-16 13:13:18.170994] Train Epoch: 4 [0/712 (0%)]	Average loss: 0.010382
[2020-02-16 13:14:20.203522] Train Epoch: 5 [0/712 (0%)]	Average loss: 0.003289
[2020-02-16 13:15:21.801463] Train Epoch: 6 [0/712 (0%)]	Average loss: 0.014083
[2020-02-16 13:16:23.037835] Train Epoch: 7 [0/712 (0%)]	Average loss: 0.001877
[2020-02-16 13:17:24.939293] Train Epoch: 8 [0/712 (0%)]	Average loss: 0.092752
[2020-02-16 13:18:26.810261] Train Epoch: 9 [0/712 (0%)]	Average loss: 0.056591
[2020-02-16 13:19:28.856546] Train Epoch: 10 [0/712 (0%)]	Average loss: 0.011266


In [23]:
resnet2 = resnet2.eval()
pred2 = []
Y2 = []
for i, (x,y) in enumerate(test_loader):
    with torch.no_grad():
        output = resnet2(x)
    pred2 += [int(l.argmax()) for l in output]
    Y2 += [int(l) for l in y]
correct2=0
for i in range(len(Y2)):
    correct2 += Y2[i]==pred2[i]
print(correct2/len(Y2))

0.8833333333333333


In [0]:
# resnet18の全パラメータ学習
resnet3 = models.resnet18()

In [28]:
for epoch in range(1, 10 + 1):
    resnet3 = train(resnet3, epoch)

[2020-02-16 13:29:18.453253] Train Epoch: 1 [0/712 (0%)]	Average loss: 0.202953
[2020-02-16 13:31:54.427071] Train Epoch: 2 [0/712 (0%)]	Average loss: 0.061018
[2020-02-16 13:34:25.779667] Train Epoch: 3 [0/712 (0%)]	Average loss: 0.032955
[2020-02-16 13:36:54.073756] Train Epoch: 4 [0/712 (0%)]	Average loss: 0.024499
[2020-02-16 13:39:22.556053] Train Epoch: 5 [0/712 (0%)]	Average loss: 0.038423
[2020-02-16 13:41:53.634684] Train Epoch: 6 [0/712 (0%)]	Average loss: 0.028786
[2020-02-16 13:44:22.268298] Train Epoch: 7 [0/712 (0%)]	Average loss: 0.026048
[2020-02-16 13:46:45.896287] Train Epoch: 8 [0/712 (0%)]	Average loss: 0.022815
[2020-02-16 13:49:08.405298] Train Epoch: 9 [0/712 (0%)]	Average loss: 0.022976
[2020-02-16 13:51:29.516705] Train Epoch: 10 [0/712 (0%)]	Average loss: 0.017979


In [29]:
# パラメータ数が多すぎると、過学習してテストデータへの精度が落ちる

resnet3 = resnet3.eval()
pred3 = []
Y3 = []
for i, (x,y) in enumerate(test_loader):
    with torch.no_grad():
        output = resnet3(x)
    pred3 += [int(l.argmax()) for l in output]
    Y3 += [int(l) for l in y]
correct3=0
for i in range(len(Y3)):
    correct3 += Y3[i]==pred3[i]
print(correct3/len(Y3))

0.45
